In [2]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [3]:
import warnings

warnings.filterwarnings( 'ignore' )

In [4]:
train_data = pd.read_csv("train.csv") 
test_data = pd.read_csv("test.csv")

In [5]:
x_data = train_data.loc[:, 'f0':'f99']
y_data = train_data.loc[:, 'loss']

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
x_train, x_test, y_train, y_test=train_test_split(x_data,
                                                  y_data,
                                                  test_size=0.2,   #전체 중 20%를 테스트용으로 분할
                                                                   #나머지 80%는 훈련용
                                                  shuffle=True,    #무작위로 섞어서 추출
                                                  random_state=20) #무작위 추출 시 일정한 기준으로

In [8]:
# catboost 참고 코드: https://catboost.ai/docs/concepts/python-usages-examples.html
# parameter tuning: https://catboost.ai/docs/concepts/parameter-tuning.html
from catboost import CatBoostRegressor
import catboost as cb

In [9]:
train_pool = cb.Pool(x_train, y_train)
test_pool = cb.Pool(x_test)

In [24]:
model = CatBoostRegressor(iterations=100,
                          learning_rate=0.3,
                          depth=7,
                          random_seed = 20,
                          l2_leaf_reg = 1,
                          loss_function='RMSE')
model.fit(train_pool)

0:	learn: 7.9403087	total: 28.9ms	remaining: 2.87s
1:	learn: 7.9327662	total: 50ms	remaining: 2.45s
2:	learn: 7.9270070	total: 73.4ms	remaining: 2.37s
3:	learn: 7.9216587	total: 94.9ms	remaining: 2.28s
4:	learn: 7.9178865	total: 115ms	remaining: 2.18s
5:	learn: 7.9130544	total: 137ms	remaining: 2.14s
6:	learn: 7.9091847	total: 158ms	remaining: 2.1s
7:	learn: 7.9055441	total: 182ms	remaining: 2.09s
8:	learn: 7.9006902	total: 205ms	remaining: 2.07s
9:	learn: 7.8972596	total: 227ms	remaining: 2.04s
10:	learn: 7.8935065	total: 254ms	remaining: 2.05s
11:	learn: 7.8899580	total: 277ms	remaining: 2.03s
12:	learn: 7.8872211	total: 298ms	remaining: 1.99s
13:	learn: 7.8838608	total: 319ms	remaining: 1.96s
14:	learn: 7.8804965	total: 340ms	remaining: 1.93s
15:	learn: 7.8776335	total: 361ms	remaining: 1.9s
16:	learn: 7.8746108	total: 382ms	remaining: 1.86s
17:	learn: 7.8713159	total: 405ms	remaining: 1.84s
18:	learn: 7.8681443	total: 427ms	remaining: 1.82s
19:	learn: 7.8648216	total: 447ms	remaini

In [25]:
from sklearn.metrics import mean_squared_error
preds = model.predict(test_pool)
rms = np.sqrt(mean_squared_error(y_test, preds))
print(rms)

7.830323815954036


In [68]:
# depth = 6

# model = CatBoostRegressor(iterations=10000,
#                           learning_rate=0.03,
#                           depth=6,
#                           random_seed = 20,
#                           early_stopping_rounds = 5,
#                           loss_function='RMSE')
# rmse:7.813845889737223
# score:7.90029

# iter: 50000, learning_rate = 0.03
# rmse: 7.923826522022192

# iter: 5000, learning_rate = 0.03
# rmse: 7.799099906546834
# score: 7.89085

# iter: 3000, leaning_rate = 0.03 
# rmse: 7.796606801724997

# iter: 2000, learning_rate = 0.03
# rmse: 7.7980331449936235

# iter: 3000, learning_rate = 0.05
# rmse: 7.807504189243877

# iter: 3000, learning_rate = 0.01
# rmse: 7.807666005260566

# iter: 4000, learning_rate = 0.01
# rmse: 7.807666005260566

# iter: 7000, learning_rate = 0.01
# rmse: 7.7954222999385046

# iter: 10000, learning_rate = 0.01
# rmse: 7.793435521881254

# iter: 20000, learning_rate = 0.01
# rmse: 7.793909233479778

# iter: 10000, learning_rate = 0.01, L2 regularization
# rmse: 7.793327223673384

# depth = 8

# iter: 10000, learning_rate = 0.01, L2 regularization (best)
# rmse: 7.791979454044701

# iter: 8000, learning_rate = 0.01, L2 regularization
# rmse: 7.793435521881254

# depth = 10
# iter: 10000, learning_rate = 0.01, L2 regularization
# rmse: 7.7976

# iter: 100, learning_rate = 0.3, L2 regularization
# rmse: 7.830323815954036




In [90]:
new_x_test = test_data.loc[:, 'f0':'f99']
new_y_pred =  model.predict(new_x_test)


from pandas import Series, DataFrame

raw_data = {'id': [ i for i in range(250000,400000)],
            'loss': new_y_pred }

data = DataFrame(raw_data)
data.set_index('id', inplace=True)
print(data)
data.to_csv("submission.csv", mode='w')

            loss
id              
250000  8.134695
250001  4.437273
250002  8.520938
250003  7.183588
250004  7.617621
...          ...
399995  7.778029
399996  7.007706
399997  5.811241
399998  5.751366
399999  6.511879

[150000 rows x 1 columns]
